# Google Top Stories - An exploration

This dataset was collected during the pandemic, it contains headlines shown by Google Search.

The dataset is organized by year, month, date, hour, in a series of nested folders. The leaf folders contain JSON files with information about top stories. 

Your goal is to write first Python function that takes the name of the folder "GoogleTopStories" and stores the content of all the JSON files into a single CSV file. Once you have a CSV file, you can practice your pandas & seaborn skills in analyzing the data.

Some tips and ideas for analysis:
1. Use the Python command `os.walk` to iteratively traverse the nested subfolders. Account for hidden files that start with .
2. Although the dataset has a column domain, imagine that it doesn't have it and write a function that takes a URL and extract the domain name from it and saves it in a colum. Use the .apply method to create the column. Learn how to parse URLs with urllib.parse.
3. Create a bar chart with the frequencies of the top 10 domains. (Use domains, since 'source name' contains duplicates.)
4. Create a heatmap that shows the distribution of domains across the 'story_position' values.
5. Create timeseries of unique articles per month by query type. (This is challenging.)

In [1]:
import os
import json
import csv

In [2]:
def convert_to_csv(folder_path, output_file):
    # Open the CSV file for writing
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['title', 'url', 'source', 'time', 'date', 'query', 'story_position', 'domain']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        # Traverse through the folder and its subfolders
        for root, _, files in os.walk(folder_path):
            for file in files:
                # Check if the file is a JSON file
                if file.endswith('.json'):
                    file_path = os.path.join(root, file)
                    with open(file_path, 'r', encoding='utf-8') as json_file:
                        data = json.load(json_file)
                        # Iterate over each item in the JSON data
                        for item in data:
                            writer.writerow({
                                'title': item['title'],
                                'url': item['url'],
                                'source': item['source'],
                                'time': item['time'],
                                'date': item['date'],
                                'query': item['query'],
                                'story_position': item['story_position'],
                                'domain': item['domain']
                            })

In [3]:
convert_to_csv('/Users/jenni/Desktop/GoogleTopStories', 'google_top_stories.csv')
